Steps to start with colab:
* Click on the CONNECT button on right side of the notebook above.
* Select "Connect to host runtime".
* Once connected you will see RAM and Disk alloted.
* CSVs are available in Sample Data Folder which can be accessed using the Folder symbol on the left panel.
* Rest of the commands etc are same as jupyter notebook.

***Note: In case unable to connect to hosted runtime configure local runtime. Ref: https://medium.com/@jasonrichards911/getting-local-with-google-colab-a4d69f373364***

***Note: The RAM and Disk allocated after connecting to hosted runtime are upgraded automatically once the limit is reached. You will be prompted with notification just accept that and continue with you work.***

Attributes for ARM: {state, rural/urban, sex, age, weight, height, iodine salt test, haemoglobin level, systolic BP, diastolic BP, pulse rate, fasting glucose level}

Categorical: {state, rural/urban, sex}
Continuous: {age, BMI, iodine salt test, haemoglobin level, systolic BP, diastolic BP, pulse rate, fasting glucose level}

* Haemoglobin levels: CONCLUSION DEPENDS ON AGE, SEX (eg below)
    Age       Female      Male
0–30 days	13.4–19.9	13.4–19.9
31–60 days	10.7–17.1	10.7–17.1
2–3 months	9.0–14.1	9.0–14.1
3–6 months	9.5–14.1	9.5–14.1
6–12 months	11.3–14.1	11.3–14.1
1–5 years	10.9–15.0	10.9–15.0
5–11 years	11.9–15.0	11.9–15.0
11–18 years	11.9–15.0	12.7–17.7

In [0]:
##############################################################
## Comment out if running on google colab                    #
## Remember to replace the file ID accordingly if using colab#
##############################################################

# # Import PyDrive and associated libraries.
# # This only needs to be done once per notebook.
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # Download a file based on its file ID.
# # A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
# cpUP_file = '19un53B6ZHQnAeI74T3Swb85C-U5Sx4DE'
# cpUK_file = '1Q5U4BieMs5pRf0kNNaErktL4oT3zPVC0'
# cpOR_file = '1KwM5NDsyIFvpX0SwmEUcm3CtU6nXFZfZ'
# cpMP_file = '1xs30vLQ9M41Seyx6Xi6f3jeXvSezXvA1'
# cpJH_file = '1xXn0MXiRDL867Q1DPe4nWZR7Du_UnhIx'
# cpCH_file = '1086hY7rzlbnz_VSOD7Vr1pY06330PMUR'
# cpBH_file = '1JGFd98ZsfS0vQE8HBneMyWa57a0ToAWF'
# bd_file = '1HPPyENxszY_vNHoQ2kjVLs_VPKt7_BPE'
# ht_file = '1FU7WmHTWi7sMUbrqXIX_TBqND0w-TZ7h'

# dcpUP_file = drive.CreateFile({'id': cpUP_file})
# dcpUK_file = drive.CreateFile({'id': cpUK_file})
# dcpOR_file = drive.CreateFile({'id': cpOR_file})
# dcpMP_file = drive.CreateFile({'id': cpMP_file})
# dcpJH_file = drive.CreateFile({'id': cpJH_file})
# dcpBH_file = drive.CreateFile({'id': cpBH_file})
# dcpCH_file = drive.CreateFile({'id': cpCH_file})
# dbd_file =  drive.CreateFile({'id': bd_file})
# dht_file =  drive.CreateFile({'id': ht_file})

# dcpUP_file.GetContentFile('preprocessedUttarPradesh.csv')  
# dcpUK_file.GetContentFile('preprocessedUttarakhand.csv')  
# dcpOR_file.GetContentFile('preprocessedOrissa.csv')  
# dcpMP_file.GetContentFile('preprocessedMadhyaPradesh.csv')  
# dcpJH_file.GetContentFile('preprocessedJharkhand.csv')  
# dcpCH_file.GetContentFile('preprocessedChattisgarh.csv')  
# dcpBH_file.GetContentFile('preprocessedBihar.csv')
# dbd_file.GetContentFile('binarized_data.csv')
# dht_file.GetContentFile('hash_table_suppcnt.pkl')

import pandas as pd
import pickle

Calculate Lift

In [0]:
def lift(x_itemset,y_itemset,itemset,hash_table,trans_cnt):

    lift_value = (hash_table[itemset] * trans_cnt)/(hash_table[x_itemset] * hash_table[y_itemset])

    return lift_value

Calculate Confidence

In [0]:
def confidence(x_itemset,itemset,hash_table):

    confidence_value = hash_table[itemset]/hash_table[x_itemset]

    return confidence_value

Association Rule Generation

In [0]:
from itertools import chain, combinations

def generate_assoc_rules(freq_itemsets_list,hash_table,trans_cnt,min_conf,min_lift):

    #  data structure to store possible association rules efficiently
    assoc_rules = dict()

    for itemset_list in freq_itemsets_list:
        itemset = frozenset(itemset_list)
        # Entering rules X-->Y in dictionary with key as X and Y as value
        for x_itemset in list(chain.from_iterable(combinations(itemset_list,r) for r in range(1,len(itemset_list)))):
            # Frozenset removes the necessity of taking care of order of the input items in a list
            x_itemset = frozenset(x_itemset)
            y_itemset = itemset.difference(x_itemset);
            # Checking whether rules satisfy min confidence and min lift criteria
            if confidence(x_itemset,itemset,hash_table) >= min_conf and lift(x_itemset,y_itemset,itemset,hash_table,trans_cnt) >= min_lift:
                if x_itemset in assoc_rules:
                    assoc_rules[x_itemset].append(y_itemset)
                else :
                    assoc_rules[x_itemset] = [y_itemset]

    return assoc_rules

# test = [[1,2,5],[3,4],[1,4,6]]
# generate_assoc_rules(test)

A method which takes two sorted itemsets and returns a merged and sorted itemset

In [0]:
def mergeItemSets(itemSet1, itemSet2, itemList):
    
    #Sorted list of items in first itemset
    itemSortedList1=[]
    #Sorted list of items in second itemset
    itemSortedList2=[]
    #Merged sorted list of items 
    mergedList=[]
    
    #Constructing corresponding sorted list of item numbers for items in first itemset 
    for item in itemSet1:
        itemSortedList1.append(itemList.index(item))
     
    #Constructing corresponding sorted list of item numbers for items in first itemset
    for item in itemSet2:
        itemSortedList2.append(itemList.index(item))
    
    i=0
    j=0
    
    #Merging the two sorted lists as in Merge Sort
    while i<len(itemSortedList1) and j<len(itemSortedList2):
        if itemSortedList1[i]<itemSortedList2[j]:
            mergedList.append(itemSortedList1[i])
            i=i+1
        elif itemSortedList1[i]>itemSortedList2[j]:
            mergedList.append(itemSortedList2[j])
            j=j+1
        else:
            mergedList.append(itemSortedList1[i])
            mergedList.append(itemSortedList2[j])
            i=i+1
            j=j+1
            
    while i<len(itemSortedList1):
        mergedList.append(itemSortedList1[i])
        i=i+1
        
    while j<len(itemSortedList2):
        mergedList.append(itemSortedList2[j])
        j=j+1
    
    #Reverting the sorted list of item numbers to a sorted list of items
    for i in range(0, len(mergedList)):
        mergedList[i]=itemList[mergedList[i]]
        
    return mergedList

Calculate Support Count

In [0]:
def supp_count(hash_table,itemset_list):

    itemset = frozenset(itemset_list)

    if itemset in hash_table :
        return hash_table[itemset]

    return 0

A method which takes two lists and checks if they share a prefix of a given length

In [0]:
def haveCommonPrefix(list1, list2, prefixLength):
    
    answer=True
    
    for i in range(0, prefixLength):
        if list2[i]!=list1[i]:
            answer=False
            break
            
    return answer

A method which takes a list of itemsets and returns it after eliminating all infrequent itemsets

In [0]:
def elimInfreqCandItemSets(prunedCandItemSets, minSup, hashTable):
    
    #List of all frequent K itemsets 
    KItemSets=[]
    
    #Eliminating each infrequent K-itemset
    for itemSet in prunedCandItemSets:
        supCount=supp_count(hashTable, itemSet)
        if supCount>=minSup:
            KItemSets.append(itemSet)
             
    return KItemSets

A method which generates all subsets of an itemset, having length one less than that of the itemset

In [0]:
import itertools

def generateSubsetsOfItemSet(itemSet):
    return list(itertools.combinations(set(itemSet), len(itemSet)-1)) 

A method which takes a list of itemsets and returns it after eliminating the itemsets having infrequent subsets, of size one less than that of the itemset that is, prunes the list of itemsets

In [0]:
def pruneCandItemSets(candItemSets, minSup, hashTable):
    
    #List of pruned candidate K-itemsets
    prunedCandItemSets=[]
    
    #Pruning each cadidate itemset if it has atleast one subset (sized K-1) which is infrequent
    for itemSet in candItemSets:
        pruneItemSet=0
        #Generating K-1 subsets of itemset
        subsets=generateSubsetsOfItemSet(itemSet)
        #Checking support count of each subset
        for subset in subsets:
            supCount=supp_count(hashTable, list(subset))
            #Pruning itemset if subset is infrequent
            if supCount<minSup:
                pruneItemSet=1
                break
        if pruneItemSet==0:
            prunedCandItemSets.append(itemSet)
            
    return prunedCandItemSets

A method which generates candidate itemsets of a given size from the frequent itemsets of one lesser size

In [0]:
def generateCandItemSets(K, prevSetOfFreqItemSets, itemList):
    
    #List of generated candidate K-itemsets
    candItemSets=[]
    
    #Merging pairs of K-1 itemsets which share a K-2 prefix
    for itemSet1Index in range(0, len(prevSetOfFreqItemSets)):
        for itemSet2Index in range(itemSet1Index+1, len(prevSetOfFreqItemSets)):
            #For K=2 
            if K==2:
                candItemSets.append(mergeItemSets(prevSetOfFreqItemSets[itemSet1Index], prevSetOfFreqItemSets[itemSet2Index], itemList))
            #For K>2 check for common K-2 prefix
            elif haveCommonPrefix(prevSetOfFreqItemSets[itemSet1Index], prevSetOfFreqItemSets[itemSet2Index], K-2):
                prefix=prevSetOfFreqItemSets[itemSet1Index][:K-2]
                candItemSets.append(prefix+mergeItemSets(prevSetOfFreqItemSets[itemSet1Index][K-2:], prevSetOfFreqItemSets[itemSet2Index][K-2:], itemList))
            
    return candItemSets        

A method which generates frequent itemsets of a given size

In [0]:
def generateFreqKItemSets(K, minSup, prevSetOfFreqItemSets, itemList, hashTable):
    
    #List of K-itemsets for given minSup
    freqKItemSets=[]
    
    #Generating frequent K-itemsets
    
    #For K=1
    if K==1:
        for attr in itemList:
            supCount=supp_count(hashTable, [attr])
            if supCount>=minSup:
                freqKItemSets.append([attr])
    #For K>1
    else:
        #Candidate generation
        candItemSets=generateCandItemSets(K, prevSetOfFreqItemSets, itemList)
        #Candidate pruning
        prunedCandItemSets=pruneCandItemSets(candItemSets, minSup, hashTable)
        #Candidate elimination
        freqKItemSets=elimInfreqCandItemSets(prunedCandItemSets, minSup, hashTable)
        
    return freqKItemSets

A method which generates frequent itemsets of all sizes for a given list of itemsets

In [0]:
def generateFreqItemSets(itemList, minSup, hashTable):
    
    #Maximum size of frequent itemset which can be generated
    maxSizeOfItemSet=len(itemList)
    #List of all frequent itemsets for given minSup
    freqItemSets=[]
    #List of all K-itemsets for given minSup
    KItemSets=[]
    
    #Generating frequent K-itemsets
    for K in range(1, maxSizeOfItemSet):
        KItemSets=generateFreqKItemSets(K, minSup, KItemSets, itemList, hashTable)
        freqItemSets = freqItemSets + KItemSets
        #Stopping criterion
        if len(KItemSets)<=1:
            break
        
    return freqItemSets

A driver methods which uses the methods above to generate frequent itemsets and corresponding association rules for all minimum support values given to it as input 

In [0]:
import math

def performAssocRuleMining_freqItemsets(itemList, minSupFractions, numOfTransac, hashTable):
    
    #Generating frequent itemsets for each minSup value
    for minSupFraction in minSupFractions:
        minSup=math.ceil(minSupFraction*numOfTransac)
        freqItemSets=generateFreqItemSets(itemList, minSup, hashTable)
        
        fn = "../Data/Association Rule Mining Data/Frequent Itemsets/freq_itemsets_ms"+str(minSupFraction)+".pkl"
        wf = open(fn,"wb")
        pickle.dump(freqItemSets, wf)
        wf.close()
    

In [0]:
def performAssocRuleMining_assocRules(freqItemSets,hashTable,transCnt,min_conf,min_lift):

    #List of association rules for all minimum support values put together
    assocRulesForAllMinSup=[]
    
    # Generate association rules for minSup value
    assocRulesForAllMinSup.append(generate_assoc_rules(freqItemSets, hashTable,transCnt, min_conf,min_lift)) 

    return assocRulesForAllMinSup

Hash Table Generation

In [0]:
import pandas as pd
from itertools import chain,combinations

def generate_hash_table(df):

    hash_table = dict()
    row_count,col_count = df.shape
    columns = [col for col in df.columns]

    for row_no in range(row_count):
        # Making the list of all the columns having value 1 for a data record.
        transaction  = [columns[col_no] for col_no in range(col_count) if df.iloc[row_no][col_no]==1]
        # Generating all subsets of the columns having value 1 for a data record.
        for itemset in list(chain.from_iterable(combinations(transaction,r) for r in range(1,len(transaction)+1))):
            itemset = frozenset(itemset)
            # Check if item is already in the hash table else add it.
            if itemset in hash_table:
                hash_table[itemset] += 1
            else :
                hash_table[itemset] = 1

    return hash_table

# test = {'a':[0,1],'b':[1,0],'c':[1,1],'d':[1,1]}
# df = pd.DataFrame.from_dict(test)
# hash_table = generate_hash_table(df)
# print(supp_count(hash_table,['x','p']))
# generate_assoc_rules([['x','z','p']],hash_table,0.4)

A method which handles categorical attributes of the given dataframe by binarizing them

In [0]:
def handleCategoricalAttr(categoricalAttr, dataFrame):
    
    #Binarization of each categorical attribute
    for attr in categoricalAttr:
        dataFrame=pd.get_dummies(dataFrame, columns=[attr], prefix=[attr])
        
    return dataFrame

A method which handles continuous attributes of the given dataframe by discretizing them according to the given intervals and subsequently binarizing them

In [0]:
def handleContinuousAttr(contAttr, bins, dataFrame):
    
    #Discretization of each continuous attribute
    for i in range(0, len(contAttr)):
        dataFrame[contAttr[i]]=pd.cut(dataFrame[contAttr[i]], bins[i], right=False, labels=list(str(num) for num in range(1,len(bins[i]))))
        
    #Binarization of each discretized attribute
    dataFrame=handleCategoricalAttr(contAttr, dataFrame)
    
    return dataFrame

A driver method which uses the above methods to handle categorical and continuous attributes of a dataframe

In [0]:
def handleAttr(dataFrame):
    
    #List of categorical attributes
    categoricalAttr=['state_code', 'rural_urban', 'Sex']
    #List of continuous attributes
    contAttr=['Age', 'BMI', 'test_salt_iodine', 'Haemoglobin_level', 'BP_systolic', 'BP_Diastolic', 'Pulse_rate', 'fasting_blood_glucose_mg_dl']
    #List of intervals for discretization of continuous attributes
    bins=[[1, 20, 40, 80, 100], [1, 18.5, 25, 30, 100], [0, 15, 31], [1, 9, 10, 11, 12, 13, 20], [0, 90, 120, 130, 140, 181, 200], [0, 60, 80, 90, 121, 150], [0, 40, 55, 60, 75, 80, 150], [50, 70, 100, 126, 350]]#list of lists filled in with domain knowledge
    
    #Handling categorical attributes
    dataFrame=handleCategoricalAttr(categoricalAttr, dataFrame)
    #Handling continuous attributes
    dataFrame=handleContinuousAttr(contAttr, bins, dataFrame)
    
    return dataFrame

Preparing for Association Rules Mining

In [0]:
def initiating_ARM():
    
    #Defining list containing the names of all states
    stateNames=["Bihar", "Chattisgarh", "Jharkhand", "MadhyaPradesh", "Orissa", "Uttarakhand", "UttarPradesh"]
    
    #Defining the file path prefix
    pathPrefix="../Data/Preprocessed Data/preprocessed"
    
    #Creating an empty dataframe
    dataFrame=pd.DataFrame()

    #Concatenating preprocessed data from all states
    for state in stateNames:
        dataFrame=dataFrame.append(pd.read_csv(pathPrefix+state+'.csv', low_memory=False), ignore_index=True)
        
    #Keeping only the attributes relevant for association rule mining
    dataFrame=pd.DataFrame(dataFrame, columns=['state_code', 'rural_urban', 'Sex', 'Age', 'BMI', 'test_salt_iodine', 'Haemoglobin_level', 'BP_systolic', 'BP_Diastolic', 'Pulse_rate', 'fasting_blood_glucose_mg_dl'])
    
    #Handling categorical and continuous attributes
    dataFrame=handleAttr(dataFrame)
        
    #Generating the hashtable for support of each itemset
    hashTable=generate_hash_table(dataFrame)

    #Store both hash table and binarized data to avoid re-executions
    dataFrame.to_csv('../Data/Association Rule Mining Data/binarized_data.csv',index=False)

    wf = open("../Data/Association Rule Mining Data/hash_table_suppcnt.pkl","wb")
    pickle.dump(hashTable,wf)
    wf.close()

Driver functions to perform Association Rules Mining

In [0]:
def execute_ARM_genFreqItemsets():

    # Reading binarized data in dataframe
    dataFrame = pd.read_csv('../Data/Association Rule Mining Data/binarized_data.csv')
    
    # Reading hash table containing support count from csv
    hashTable = pickle.load(open("../Data/Association Rule Mining Data/hash_table_suppcnt.pkl", "rb"))
    
    # Creating the list of 'items' that is, the list of columns in the dataframe
    itemList=[]
    for colIndex in range(0, len(dataFrame.columns)):
        itemList.append(dataFrame.columns[colIndex])
    
    # List of minSup values that the user wants to experiment with (in fractions)
    minSupFractions=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    #Generating itemsets for all minSup values
    performAssocRuleMining_freqItemsets(itemList, minSupFractions, dataFrame.shape[0], hashTable)

In [0]:
import os

def execute_ARM_genAssocRules():

    # All possible value of min support, min confidence and min lift values
    minSupFraction=[0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    minConfFraction=[0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    minLiftFraction=[0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

    # Reading binarized data in dataframe
    dataFrame = pd.read_csv('../Data/Association Rule Mining Data/binarized_data.csv')
    transCnt = dataFrame.shape[0]

    # Reading hash table containing support count from csv
    hashTable = pickle.load(open("../Data/Association Rule Mining Data/hash_table_suppcnt.pkl", "rb"))

    for minSup in minSupFraction:

        fn = "../Data/Association Rule Mining Data/Frequent Itemsets/freq_itemsets_ms"+str(minSup)+".pkl"
        freqItemsets = pickle.load(open(fn,"rb"))

        for minConf in minConfFraction: 
            for minLift in minLiftFraction: 

                #Generating association rules for minSup,min values
                assocRules=performAssocRuleMining_assocRules(freqItemsets, hashTable, transCnt,minConf, minLift)

                fn = '../Data/Association Rule Mining Data/Association_Rules/assoc_rules_ms('+str(minSup)+')_mc('+str(minConf)+')_ml('+str(minLift)+').txt'
                rules_file = open(fn, 'w') 
                
                #Printing association rules for all minSup values
                print('Association Rules:')
                rulecnt = 0
                print("Min Sup Fraction = "+str(minSup)+"%\n",file = rules_file)
                print("Min Conf Fraction= "+str(minConf)+"%\n",file = rules_file)
                print("Min Lift Fraction = "+str(minLift)+"%\n",file = rules_file)
                for key,value in assocRules[0].items():
                    rulecnt = rulecnt + len(value)
                    for y in value:
                        print(str(set(key))+" --> "+str(set(y)),file = rules_file)

                print(rulecnt)

                rules_file.close()

                # remove the files with no association rules in them 
                if rulecnt == 0 :
                    os.remove(fn)
                    

Main functions

In [0]:
# To avoid unecessary request to create support hash table and binarized data
# Need to be run only when running for first time and dont have binarized data, hash_table file
initiating_ARM()

In [0]:
# Main driver program to generate frequent itemsets
# Need to be run only when running for first time and you dont have min_support files 
execute_ARM_genFreqItemsets()

In [0]:
# Main driver program to generate association rules
execute_ARM_genAssocRules()